### Take a list of images (jpg) + yolo labels and apply Albumentations augmentations

to generate new images correctly labeles (with BB in Yolo v5 format)

In [1]:
from os import path
import glob
import cv2
import albumentations as A
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
from common_functions import read_bb, write_bb, yolo_to_cv2, do_check

In [3]:
# here we have to define a working way taking into account how the old_name structure is
# in this test the files are from Roboflow, so there are lots of "."
# you can find another way
def new_name(orig_name):
    # taking the first split.. it is one of the possible choices
    new_name_without_ext = orig_name.split(".")[0]

    return new_name_without_ext

In [4]:
DATASET_BASE_DIR = "/Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/train"
# under this dir I have images and labels

AUG_DEST_BASE = "/Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train"

IMAGE_DEST_DIR = (
    "/Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images"
)
LABELS_DEST_DIR = (
    "/Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/labels"
)

images_list = sorted(glob.glob(path.join(DATASET_BASE_DIR, "images/*.jpg")))
labels_list = sorted(glob.glob(path.join(DATASET_BASE_DIR, "labels/*.txt")))

#### Augmentations

In [5]:
# we have to carefully consider which trasnformation we're putting here.
# the class_num MUST not be affected...

transform = A.Compose(
    [A.Rotate(limit=90), A.HorizontalFlip(p=0.5)],
    bbox_params=A.BboxParams(format="yolo"),
)

In [8]:
# process in loop
HOW_MANY_NEW_IMAGES = 10

tot_new_images = 0

for img_path_name, labels_path_name in zip(images_list, labels_list):
    # remove path
    img_only_name = img_path_name.split("/")[-1]
    labels_only_name = labels_path_name.split("/")[-1]

    print(f"Processing {img_only_name}, {labels_only_name}")

    for i in range(HOW_MANY_NEW_IMAGES):
        # every time Albumentations generate a new img + bb
        # build file names
        i += 1
        new_name_jpg = new_name(img_only_name) + f"_{i}.jpg"
        new_name_txt = new_name(img_only_name) + f"_{i}.txt"
        
        # read old
        img = cv2.imread(img_path_name)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        yolo_bb_list = read_bb(labels_path_name)
        
        # create new
        transformed = transform(image=img_rgb, bboxes=yolo_bb_list)
        transformed_image = transformed["image"]
        transformed_bboxes = transformed["bboxes"]
        
        do_check(yolo_bb_list, transformed_bboxes)
        
        print(f"Saving {new_name_jpg} in {IMAGE_DEST_DIR}")
        img_bgr = cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(path.join(IMAGE_DEST_DIR, new_name_jpg), img_bgr)
        
        print(f"Saving {new_name_txt} in {LABELS_DEST_DIR}")
        write_bb(path.join(LABELS_DEST_DIR, new_name_txt), transformed_bboxes)
        
        tot_new_images += 1
        
    print()
    print()

Processing 00222_clear_jpg.rf.83b62755045b4f2824f818dfbc82c203.jpg, 00222_clear_jpg.rf.83b62755045b4f2824f818dfbc82c203.txt
Saving 00222_clear_jpg_1.jpg in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images
Saving 00222_clear_jpg_1.txt in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/labels
Saving 00222_clear_jpg_2.jpg in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images
Saving 00222_clear_jpg_2.txt in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/labels
Saving 00222_clear_jpg_3.jpg in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images
Saving 00222_clear_jpg_3.txt in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/labels
Saving 00222_clear_jpg_4.jpg in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train/images
Saving 00222_clear_jpg_4.txt in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_t

In [9]:
print(f"We have generated {tot_new_images} images annotated with BB in {AUG_DEST_BASE} directory.")

We have generated 60 images annotated with BB in /Users/lsaetta/Progetti/yolo_augmentations/yolo_dataset/augmented_train directory.
